In [ ]:
import numpy as np
import os
import pandas as pd
import pathlib
import skimage.exposure as ex
import skimage.io as io
import torch
import torch.optim as optim

In [ ]:
!pip install haroun==0.0.4

# import my Library (Pytorch Framework)
from haroun import Data
from haroun import Model
from haroun.losses import RMSE

In [ ]:
def load_dir(path):
    # Define data paths
    data_path = path.parent / "input" / "france-ironcar-competition" 
    images_path = data_path / "images"

    # Read labels and store them in pandas dataframe
    df = pd.read_csv(f"{data_path}/labels.csv")
    df = df.values.tolist()

    # Load Data
    images = []
    labels = []

    for value in df:
        file, label = value
        image = io.imread(f"{images_path}/{file}")
        images.append(image)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels



In [ ]:
def flip(images, labels, axis): 
    flipped_images = np.flip(images, axis)
    flipped_labels = [4 - label for label in labels]
    flipped_labels = np.array(flipped_labels)
    return flipped_images,flipped_labels

def brightness(images, labels, gamma):
    brightness_images =  np.array([ex.adjust_gamma(image, gamma ,gain=1) for image in images])
    brightness_labels = labels
    return brightness_images, brightness_labels

def data_augmentation(images, labels):
    # Data augmentation (flip_horizontal)
    flipped_y_images,flipped_y_labels = flip(images, labels, axis=2)
    
    # Concatenate arrays
    images = np.concatenate([images, flipped_y_images])
    labels = np.concatenate([labels, flipped_y_labels])
    
    # Data augmentation (brightness)
    brighten_images, brighten_labels = brightness(images, labels, gamma=0.5)
    darken_images, darken_labels = brightness(images, labels, gamma=1.5)
    
    # Concatenate arrays
    images = np.concatenate([images, brighten_images, darken_images])
    labels = np.concatenate([labels, brighten_labels, darken_labels])
    
    return images, labels

In [ ]:
def load_data():
    path = pathlib.Path.cwd()

    # Load data
    images, labels = load_dir(path)
    # Data augmentation
    images, labels = data_augmentation(images, labels)
    labels = np.array([["hard_left", "left", "straight", "right" ,"hard_right"][label] for label in labels])
    return images, labels

In [ ]:
# Define CNN 
class conv_layer(torch.nn.Module):

    def __init__(self, in_features, out_features):
        super(conv_layer, self).__init__()
        self.conv = torch.nn.Conv2d(in_features, out_features, kernel_size=3, stride=1, padding=1)
        self.relu = torch.nn.ReLU()
        self.norm = torch.nn.BatchNorm2d(out_features)


    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.conv(X)
        X = self.relu(X)
        X = self.norm(X)
        return X


class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.input_norm = torch.nn.BatchNorm2d(3, affine=False)
        self.layer1 = conv_layer(in_features=3, out_features=8)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.layer2 = conv_layer(in_features=8, out_features=8)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.layer3 = conv_layer(in_features=8, out_features=16)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=1, stride=2)

        self.layer4 = conv_layer(in_features=16, out_features=16)
        self.pool4 = torch.nn.MaxPool2d(kernel_size=(1,2), stride=(1,2))
        
        self.layer5 = conv_layer(in_features=16, out_features=32)
        self.pool5 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.layer6 = conv_layer(in_features=32, out_features=32)
        self.pool6 =  torch.nn.MaxPool2d(kernel_size=1, stride=2)
        
        self.layer7 = conv_layer(in_features=32, out_features=64)
        self.pool7 =  torch.nn.MaxPool2d(kernel_size=(1,2), stride=2)
        
        self.layer8 = conv_layer(in_features=64, out_features=64)
        self.pool8 =  torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        

        self.net = torch.nn.Sequential(self.layer1, self.pool1,
                                       self.layer2, self.pool2,
                                       self.layer3, self.pool3,
                                       self.layer4, self.pool4,
                                       self.layer5, self.pool5,
                                       self.layer6, self.pool6,
                                       self.layer7, self.pool7,
                                       self.layer8, self.pool8
                                       )
    
        
        self.fc1 = torch.nn.Linear(in_features=64, out_features=32)
        self.bn1 = torch.nn.BatchNorm1d(32)
        
        self.fc2 = torch.nn.Linear(in_features=32, out_features=16)
        self.bn2 = torch.nn.BatchNorm1d(16)

        self.fc3 = torch.nn.Linear(in_features=16, out_features=8)
        self.bn3 = torch.nn.BatchNorm1d(8)
        
        self.fc4 = torch.nn.Linear(in_features=8, out_features=5)
        
        self.lin = torch.nn.Sequential(self.fc1, self.bn1, self.fc2, self.bn2,
                                       self.fc3, self.bn3, self.fc4)  


    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.input_norm(X)
        X = self.net(X)
        X = X.reshape(X.size(0), -1)
        X = self.lin(X)
        X = torch.nn.functional.softmax(X, dim=1)
        return X

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = {"hard_left": 0, "left": 1, "straight": 2, "right": 3, "hard_right": 4}

data = Data(loader=load_data(), classes=classes)
data.shape()
data.show()

In [ ]:
data.dataset(split_size=0.1, shuffle=True, random_state=98,
             images_format=torch.float32, labels_format=torch.float32,
             permute=True, device=device)

In [ ]:
net = Network()
optimizer = optim.Adam(net.parameters(), lr=1.0E-3)

IronCarClassifier = Model(net, optimizer, RMSE, device)
IronCarClassifier.train(train_data=(data.train_inputs, data.train_outputs),
                        val_data=(data.val_inputs, data.val_outputs),
                        epochs=200, patience=20, batch_size=100)
IronCarClassifier.evaluate(test_data=(data.test_inputs, data.test_outputs))

In [ ]:
IronCarClassifier.plot()
IronCarClassifier.save(path="./", checkpoint_name="module")